<a href="https://colab.research.google.com/github/MrSimple07/MachineLearning_Practice/blob/main/KMU_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gpt-2-simple

  Preparing metadata (setup.py) ... done
  Created wheel for gpt-2-simple: filename=gpt_2_simple-0.8.1-py3-none-any.whl size=24557 sha256=a5984eee90bb00a96c180f31e87a8d715d4fc58294d41e5ff8c47126f65bba79
  Stored in directory: /root/.cache/pip/wheels/df/6a/fe/10d3223f78d1ac3e4c83bb4c5e2d28dfb1789c2fb4cc7ea8d0
Successfully built gpt-2-simple


In [ ]:
import gpt_2_simple as gpt2
from datetime import datetime

